In [141]:
# all imports
import numpy as np
import os
import requests
from astropy.table import Table, vstack
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.wcs import WCS
import math
import matplotlib as mpl
from astropy.visualization.wcsaxes import SphericalCircle
import pandas as pd

In [142]:
# define directories
data_dir = '/global/cfs/projectdirs/desi/science/td/pv/tfgalaxies/'
cache_dir = '/pscratch/sd/j/jjpim/cache/' # my cache path
vi_dir = '/pscratch/sd/j/jjpim/visual_inspection/' # vi image path

#constants
c = 299792.458

In [162]:
# load updated fuji table
tf_fuji = Table.read(data_dir + 'SV/desi_pv_tf_fuji_healpix_rotcurve_EOA_VI.fits') 
# load iron table
tf_iron = Table.read(data_dir + 'Y1/desi_pv_tf_iron_healpix_rotcurve_EOA_VI.fits')
#combine
tf_mastertable = vstack([tf_fuji,tf_iron])

In [158]:
#mastertable columns
tf_mastertable['VELOCITY']=np.nan
tf_mastertable['VERR']=np.nan
tf_mastertable['WEIGHTED_CENTER']=np.nan

In [164]:
#create new tables for fuji
tf_fuji['VELOCITY']=np.nan
tf_fuji['VERR']=np.nan
tf_fuji['WEIGHTED_CENTER']=np.nan

#create new tables for iron
tf_iron['VELOCITY']=np.nan
tf_iron['VERR']=np.nan
tf_iron['WEIGHTED_CENTER']=np.nan

In [159]:
# set up dictionary of unique SGA IDs
tf_fuji_dict = {}
for sga_id in np.unique(tf_fuji['SGA_ID']):
    tf_fuji_dict[sga_id] = np.where(tf_fuji['SGA_ID'] == sga_id)[0]

tf_iron_dict = {}
for sga_id in np.unique(tf_iron['SGA_ID']):
    tf_iron_dict[sga_id] = np.where(tf_iron['SGA_ID'] == sga_id)[0]

tf_mastertable_dict = {}
for sga_id in np.unique(tf_mastertable['SGA_ID']):
    tf_mastertable_dict[sga_id] = np.where(tf_mastertable['SGA_ID'] == sga_id)[0]

Below is one loop with both data sets seperated, getting key error when it runs.

In [166]:
# loop through each SGA ID
for sga_id in tf_fuji_dict or tf_iron_dict:

    #identify z values of targets
    z_targets_fuji=tf_fuji['Z'][tf_fuji_dict[sga_id]]
    z_targets_iron=tf_iron['Z'][tf_iron_dict[sga_id]]
    
    #identify good centers
    fuji_centers_b = tf_fuji['SKY_FIBER_DIST_R26'][tf_fuji_dict[sga_id]] < 0.001
    fuji_good_b = (tf_fuji['ZWARN'][tf_fuji_dict[sga_id]]==0) & (tf_fuji['DELTACHI2'][tf_fuji_dict[sga_id]]>25)
    fuji_good_centers = tf_fuji[tf_fuji_dict[sga_id]][fuji_centers_b & fuji_good_b] 
    
    iron_centers_b = tf_iron['SKY_FIBER_DIST_R26'][tf_iron_dict[sga_id]] < 0.001
    iron_good_b = (tf_iron['ZWARN'][tf_iron_dict[sga_id]]==0) & (tf_iron['DELTACHI2'][tf_iron_dict[sga_id]]>25)
    iron_good_centers = tf_iron[tf_iron_dict[sga_id]][iron_centers_b & iron_good_b] 
    
    #defining terms
    if len(fuji_good_centers) > 0:
        z_fuji = fuji_good_centers['Z']
        z_fuji_err = fuji_good_centers['ZERR']
        w_fuji = np.sum(1 / (z_fuji_err**2))
    else:
        z_fuji = np.nan * np.ones(1)
        z_fuji_err = np.nan * np.ones(1)
        w_fuji = 0
    if len(iron_good_centers) > 0:
        z_iron = iron_good_centers['Z']
        z_iron_err = iron_good_centers['ZERR']
        w_iron = np.sum(1 / (z_iron_err**2))
    else:
        z_iron = np.nan * np.ones(1)
        z_iron_err = np.nan * np.ones(1)
        w_iron = 0
    
    # calculating the weights
    w = np.sum(w_fuji + w_iron)
    
    #combine redshifts for weighted center
    z = np.sum(z_fuji + z_iron)
    
    # calculating the weighted average for center z naught
    #come back to this, looks like w cancels out / equation is wrong?
    z_o = (np.nanmean(w * z)) / w
        
    #update z naught in column
    tf_fuji['WEIGHTED_CENTER'][tf_fuji_dict[sga_id]]=z_o
    tf_iron['WEIGHTED_CENTER'][tf_iron_dict[sga_id]]=z_o
    
    #define z naught error
    z_o_err = np.sqrt(1 / w)
    
    #add column for z_o error?
    
    #find delta z from z naught and z values of all viable targets 
    delta_z_fuji = z_targets_fuji - z_o
    delta_z_iron = z_targets_iron - z_o
    
    #use delta z to find velocity
    velocity_fuji = c * delta_z_fuji
    velocity_iron = c * delta_z_iron

    # update the original tables with velocities
    tf_fuji['VELOCITY'][tf_fuji_dict[sga_id]]=velocity_fuji
    tf_iron['VELOCITY'][tf_iron_dict[sga_id]]=velocity_iron
    
    #identify z error for targets (used in v error calculation)
    z_err_fuji = tf_fuji['ZERR'][tf_fuji_dict[sga_id]]
    z_err_iron = tf_iron['ZERR'][tf_iron_dict[sga_id]]
    
    #find v error for each target
    if len(velocity_fuji) > 0:
        v_error_f = ((z_o_err**2) + (z_err_fuji**2))
        v_err_fuji=c*(np.sqrt(v_error_f))
    else:
        v_error_f = np.nan * np.ones(1)
        v_err_fuji = 0
        
    if len(velocity_iron) > 0:
        v_error_i = ((z_o_err**2) + (z_err_iron**2))
        v_err_iron = c*(np.sqrt(v_error_i))
    else:
        v_error_i = np.nan * np.ones(1)
        v_err_iron = 0
    
    # put v error into new column
    tf_fuji['VERR'][tf_fuji_dict[sga_id]]=v_err_fuji
    tf_iron['VERR'][tf_iron_dict[sga_id]]=v_err_iron

/tmp/ipykernel_2267510/962789329.py:43: RuntimeWarning: Mean of empty slice
  z_o = (np.nanmean(w * z)) / w
/tmp/ipykernel_2267510/962789329.py:50: RuntimeWarning: divide by zero encountered in long_scalars
  z_o_err = np.sqrt(1 / w)


KeyError: 9603

Planned to seperate into two different for loops (fuji one is below), but getting key error still. looks like because the for loop is referencing tf fuji dictionary I can't reference iron at all?

In [167]:
# loop through each SGA ID
for sga_id in tf_fuji_dict:

    #identify z values of targets
    z_targets_fuji=tf_fuji['Z'][tf_fuji_dict[sga_id]]
    
    #identify good centers
    fuji_centers_b = tf_fuji['SKY_FIBER_DIST_R26'][tf_fuji_dict[sga_id]] < 0.001
    fuji_good_b = (tf_fuji['ZWARN'][tf_fuji_dict[sga_id]]==0) & (tf_fuji['DELTACHI2'][tf_fuji_dict[sga_id]]>25)
    fuji_good_centers = tf_fuji[tf_fuji_dict[sga_id]][fuji_centers_b & fuji_good_b] 
    
    iron_centers_b = tf_iron['SKY_FIBER_DIST_R26'][tf_iron_dict[sga_id]] < 0.001
    iron_good_b = (tf_iron['ZWARN'][tf_iron_dict[sga_id]]==0) & (tf_iron['DELTACHI2'][tf_iron_dict[sga_id]]>25)
    iron_good_centers = tf_iron[tf_iron_dict[sga_id]][iron_centers_b & iron_good_b] 
    
    #defining terms
    if len(fuji_good_centers) > 0:
        z_fuji = fuji_good_centers['Z']
        z_fuji_err = fuji_good_centers['ZERR']
        w_fuji = np.sum(1 / (z_fuji_err**2))
    else:
        z_fuji = np.nan * np.ones(1)
        z_fuji_err = np.nan * np.ones(1)
        w_fuji = 0
    if len(iron_good_centers) > 0:
        z_iron = iron_good_centers['Z']
        z_iron_err = iron_good_centers['ZERR']
        w_iron = np.sum(1 / (z_iron_err**2))
    else:
        z_iron = np.nan * np.ones(1)
        z_iron_err = np.nan * np.ones(1)
        w_iron = 0
    
    # calculating the weights
    w = np.sum(w_fuji + w_iron)
    
    #combine redshifts for weighted center
    z = np.sum(z_fuji + z_iron)
    
    # calculating the weighted average for center z naught
    #come back to this, looks like w cancels out / equation is wrong?
    z_o = (np.nanmean(w * z)) / w
        
    #update z naught in column
    tf_fuji['WEIGHTED_CENTER'][tf_fuji_dict[sga_id]]=z_o
    tf_iron['WEIGHTED_CENTER'][tf_iron_dict[sga_id]]=z_o
    
    #define z naught error
    z_o_err = np.sqrt(1 / w)
    
    #add column for z_o error?
    
    #find delta z from z naught and z values of all viable targets 
    delta_z_fuji = z_targets_fuji - z_o
    
    #use delta z to find velocity
    velocity_fuji = c * delta_z_fuji

    # update the original tables with velocities
    tf_fuji['VELOCITY'][tf_fuji_dict[sga_id]]=velocity_fuji
    
    #identify z error for targets (used in v error calculation)
    z_err_fuji = tf_fuji['ZERR'][tf_fuji_dict[sga_id]]
    
    #find v error for each target
    if len(velocity_fuji) > 0:
        v_error_f = ((z_o_err**2) + (z_err_fuji**2))
        v_err_fuji=c*(np.sqrt(v_error_f))
    else:
        v_error_f = np.nan * np.ones(1)
        v_err_fuji = 0
    
    # put v error into new column
    tf_fuji['VERR'][tf_fuji_dict[sga_id]]=v_err_fuji

/tmp/ipykernel_2267510/1238785897.py:42: RuntimeWarning: Mean of empty slice
  z_o = (np.nanmean(w * z)) / w
/tmp/ipykernel_2267510/1238785897.py:49: RuntimeWarning: divide by zero encountered in long_scalars
  z_o_err = np.sqrt(1 / w)


KeyError: 9603

Below is the code acheived w/ prof. douglass, uses mastertable instead of seperate data sets. works but need to be able to update fuji and iron seperately.

In [160]:
# pull out all unique SGA ID's for iron and fuji
#tf_fuji_rot = np.unique(filtered_tf_fuji['SGA_ID'])
#tf_iron_rot = np.unique(filtered_tf_iron['SGA_ID'])
# combine iron and fuji unique IDs (don't need if processing seperately?)
'''tf_galaxies = np.unique(np.concatenate((tf_fuji_rot, tf_iron_rot)))
'''
tf_galaxies = np.unique(tf_mastertable['SGA_ID'])

#define targetid
#targetid_fuji = targetid in tf_fuji_rot
#targetid_iron = targetid in tf_iron_rot

#identify targets
'''
for sga_id in tf_fuji_dict:
    targets_fuji = tf_fuji[tf_fuji_dict[sga_id]]
for sga_id in tf_iron_dict:
    targets_iron = tf_iron[tf_iron_dict[sga_id]]
'''
# loop through each SGA ID
#for sga_id in tf_fuji_dict:
for sga_id in tf_galaxies:
                            
    #identify z values of targets
    '''
        z_targets_fuji=tf_fuji['Z'][tf_fuji_dict[sga_id]]
    z_targets_iron=tf_iron['Z'][tf_iron_dict[sga_id]]
    '''
    z_targets=tf_mastertable['Z'][tf_mastertable_dict[sga_id]]
    
    #identify good centers
    '''
    fuji_centers_b = tf_fuji['SKY_FIBER_DIST_R26'][tf_fuji_dict[sga_id]] < 0.001
    fuji_good_b = (tf_fuji['ZWARN'][tf_fuji_dict[sga_id]]==0) & (tf_fuji['DELTACHI2'][tf_fuji_dict[sga_id]]>25)
    fuji_good_centers = tf_fuji[tf_fuji_dict[sga_id]][fuji_centers_b & fuji_good_b] 
    
    iron_centers_b = tf_iron['SKY_FIBER_DIST_R26'][tf_iron_dict[sga_id]] < 0.001
    iron_good_b = (tf_iron['ZWARN'][tf_iron_dict[sga_id]]==0) & (tf_iron['DELTACHI2'][tf_iron_dict[sga_id]]>25)
    iron_good_centers = tf_iron[tf_iron_dict[sga_id]][iron_centers_b & iron_good_b] 
    '''
    centers_b = tf_mastertable['SKY_FIBER_DIST_R26'][tf_mastertable_dict[sga_id]] < 0.001
    good_b = (tf_mastertable['ZWARN'][tf_mastertable_dict[sga_id]]==0) & (tf_mastertable['DELTACHI2'][tf_mastertable_dict[sga_id]]>25)
    good_centers = tf_mastertable[tf_mastertable_dict[sga_id]][centers_b & good_b] 
    
    #defining terms
    '''
    if len(fuji_good_centers) > 0:
        z_fuji = fuji_good_centers['Z']
        z_fuji_err = fuji_good_centers['ZERR']
        w_fuji = np.sum(1 / (z_fuji_err**2))
    else:
        z_fuji = np.nan * np.ones(1)
        z_fuji_err = np.nap * np.ones(1)
        w_fuji = 0
    if len(iron_good_centers) > 0:
        z_iron = iron_good_centers['Z']
        z_iron_err = iron_good_centers['ZERR']
        w_iron = np.sum(1 / (z_iron_err**2))
    else:
        z_iron = np.nan * np.ones(1)
        z_iron_err = np.nap * np.ones(1)
        w_iron = 0
    '''
    z = good_centers['Z']
    z_err_c = good_centers['ZERR']
    # calculating the weights
    #w = np.sum(w_fuji + w_iron)
    w = np.sum(1 / (z_err_c**2))
    
    # calculating the weighted average for center z naught
    z_o = (np.nanmean(w * z)) / w
        
    #update z naught in column
    tf_fuji['WEIGHTED_CENTER'][tf_fuji_dict[sga_id]]=z_o
    tf_iron['WEIGHTED_CENTER'][tf_iron_dict[sga_id]]=z_o
    
    #define z naught error
    z_o_err = np.sqrt(1 / w)
    
    #find delta z from z naught and z values of all viable targets 
    '''
    delta_z_fuji = z_targets_fuji - z_o
    delta_z_iron = z_targets_iron - z_o
    '''
    delta_z = z_targets - z_o
    
    #use delta z to find velocity
    '''
    velocity_fuji = c * delta_z_fuji
    velocity_iron = c * delta_z_iron
    '''
    velocity = c * delta_z
    # update the original tables with velocities
    '''
    tf_fuji['VELOCITY'][tf_fuji_dict[sga_id]]=velocity_fuji
    tf_iron['VELOCITY'][tf_iron_dict[sga_id]]=velocity_iron
    '''
    tf_mastertable['VELOCITY'][tf_mastertable_dict[sga_id]]=velocity
    
    #identify z error for targets (used in v error calculation)
    z_err = tf_mastertable['ZERR'][tf_mastertable_dict[sga_id]]
    #z_err = z_targets['ZERR'][tf_mastertable_dict[sga_id]] -tried this, got error ? wanted to get z error for non centers
    
    #find v error for each target
    v_error_1 = ((z_o_err**2) + (z_err**2))
    v_err=c*(np.sqrt(v_error_1))
    
    # put v error into new column
    tf_mastertable['VERR'][tf_mastertable_dict[sga_id]]=v_err

/tmp/ipykernel_2267510/1273285531.py:71: RuntimeWarning: Mean of empty slice
  z_o = (np.nanmean(w * z)) / w
/tmp/ipykernel_2267510/1273285531.py:77: RuntimeWarning: divide by zero encountered in double_scalars
  z_o_err = np.sqrt(1 / w)


Below is code to run through all sga ids in mastertable but update tf iron and tf fuji seperately. does not work.

In [169]:
# pull out all unique SGA ID's for iron and fuji
#tf_fuji_rot = np.unique(filtered_tf_fuji['SGA_ID'])
#tf_iron_rot = np.unique(filtered_tf_iron['SGA_ID'])
# combine iron and fuji unique IDs (don't need if processing seperately?)
'''tf_galaxies = np.unique(np.concatenate((tf_fuji_rot, tf_iron_rot)))
'''
tf_galaxies = np.unique(tf_mastertable['SGA_ID'])

#define targetid
#targetid_fuji = targetid in tf_fuji_rot
#targetid_iron = targetid in tf_iron_rot

#identify targets
'''
for sga_id in tf_fuji_dict:
    targets_fuji = tf_fuji[tf_fuji_dict[sga_id]]
for sga_id in tf_iron_dict:
    targets_iron = tf_iron[tf_iron_dict[sga_id]]
'''
# loop through each SGA ID
#for sga_id in tf_fuji_dict:
for sga_id in tf_galaxies:
                            
    #identify z values of targets
    '''
        z_targets_fuji=tf_fuji['Z'][tf_fuji_dict[sga_id]]
    z_targets_iron=tf_iron['Z'][tf_iron_dict[sga_id]]
    '''
    z_targets=tf_mastertable['Z'][tf_mastertable_dict[sga_id]]
    
    #identify good centers
    '''
    fuji_centers_b = tf_fuji['SKY_FIBER_DIST_R26'][tf_fuji_dict[sga_id]] < 0.001
    fuji_good_b = (tf_fuji['ZWARN'][tf_fuji_dict[sga_id]]==0) & (tf_fuji['DELTACHI2'][tf_fuji_dict[sga_id]]>25)
    fuji_good_centers = tf_fuji[tf_fuji_dict[sga_id]][fuji_centers_b & fuji_good_b] 
    
    iron_centers_b = tf_iron['SKY_FIBER_DIST_R26'][tf_iron_dict[sga_id]] < 0.001
    iron_good_b = (tf_iron['ZWARN'][tf_iron_dict[sga_id]]==0) & (tf_iron['DELTACHI2'][tf_iron_dict[sga_id]]>25)
    iron_good_centers = tf_iron[tf_iron_dict[sga_id]][iron_centers_b & iron_good_b] 
    '''
    centers_b = tf_mastertable['SKY_FIBER_DIST_R26'][tf_mastertable_dict[sga_id]] < 0.001
    good_b = (tf_mastertable['ZWARN'][tf_mastertable_dict[sga_id]]==0) & (tf_mastertable['DELTACHI2'][tf_mastertable_dict[sga_id]]>25)
    good_centers = tf_mastertable[tf_mastertable_dict[sga_id]][centers_b & good_b] 
    
    #defining terms
    '''
    if len(fuji_good_centers) > 0:
        z_fuji = fuji_good_centers['Z']
        z_fuji_err = fuji_good_centers['ZERR']
        w_fuji = np.sum(1 / (z_fuji_err**2))
    else:
        z_fuji = np.nan * np.ones(1)
        z_fuji_err = np.nap * np.ones(1)
        w_fuji = 0
    if len(iron_good_centers) > 0:
        z_iron = iron_good_centers['Z']
        z_iron_err = iron_good_centers['ZERR']
        w_iron = np.sum(1 / (z_iron_err**2))
    else:
        z_iron = np.nan * np.ones(1)
        z_iron_err = np.nap * np.ones(1)
        w_iron = 0
    '''
    z = good_centers['Z']
    z_err_c = good_centers['ZERR']
    # calculating the weights
    #w = np.sum(w_fuji + w_iron)
    w = np.sum(1 / (z_err_c**2))
    
    # calculating the weighted average for center z naught
    z_o = (np.nanmean(w * z)) / w
        
    #update z naught in columns
    for sga_id in tf_fuji:
        tf_fuji['WEIGHTED_CENTER'][tf_fuji_dict[sga_id]]=z_o
    for sga_id in tf_iron:     
        tf_iron['WEIGHTED_CENTER'][tf_iron_dict[sga_id]]=z_o
    
    #define z naught error
    z_o_err = np.sqrt(1 / w)
    
    #find delta z from z naught and z values of all viable targets 
    '''
    delta_z_fuji = z_targets_fuji - z_o
    delta_z_iron = z_targets_iron - z_o
    '''
    delta_z = z_targets - z_o
    
    #use delta z to find velocity
    '''
    velocity_fuji = c * delta_z_fuji
    velocity_iron = c * delta_z_iron
    '''
    velocity = c * delta_z
    # update the original tables with velocities
    '''
    tf_fuji['VELOCITY'][tf_fuji_dict[sga_id]]=velocity_fuji
    tf_iron['VELOCITY'][tf_iron_dict[sga_id]]=velocity_iron
    '''
    for targetid in tf_fuji:
        tf_fuji['VELOCITY'][tf_fuji_dict[sga_id]]=velocity
    for targetid in tf_iron:     
        tf_iron['VELOCITY'][tf_iron_dict[sga_id]]=velocity
    
    #identify z error for targets (used in v error calculation)
    z_err = tf_mastertable['ZERR'][tf_mastertable_dict[sga_id]]
    #z_err = z_targets['ZERR'][tf_mastertable_dict[sga_id]] -tried this, got error ? wanted to get z error for non centers
    
    #find v error for each target
    v_error_1 = ((z_o_err**2) + (z_err**2))
    v_err=c*(np.sqrt(v_error_1))
    
    # put v error into new column
    for targetid in tf_fuji:
        tf_fuji['VERR'][tf_fuji_dict[sga_id]]=v_err
    for targetid in tf_iron:     
        tf_iron['VERR'][tf_iron_dict[sga_id]]=v_err

TypeError: unhashable type: 'Row'

In [161]:
#testing output for sga id 123
tf_mastertable[tf_mastertable_dict[123]]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,OBS_IN_SV,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,ROT_CURVE,EOA,VELOCITY,VERR,VELOCITY_REL,WEIGHTED_CENTER
int64,float64,float64,int64,bytes4,float64,float64,int64,float64,bytes65,bytes3,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1071233026228227,242.564878755415,53.6763270782975,10153,sv3,0.184971788085982,4.36696398757541e-05,4,0.458266502246261,fuji/healpix/sv3/bright/101/10153/redrock-sv3-bright-10153.fits,TFT,123,242.56487875541504,53.67632707829753,1.0,0.0051345584203947876,1.0000000466304622,1.0,0.0,22486.14009337598,13.370585897277147,nan,0.1099660983385235
1071233026228228,242.564908359756,53.6746327647682,10153,sv3,1.5161235823035,9.98100168252869e-05,4,0.224537968635559,fuji/healpix/sv3/bright/101/10153/redrock-sv3-bright-10153.fits,TFT,123,242.5649083597559,53.67463276476823,1.0,0.0034401541416619126,0.6700000312419042,1.0,0.0,421555.40845295583,30.045299874011604,nan,0.1099660983385235
1071233026228231,242.565028546859,53.6677528249076,10153,sv3,0.223582059648363,6.6695997421467e-05,4,5.54895717371255,fuji/healpix/sv3/bright/101/10153/redrock-sv3-bright-10153.fits,TFT,123,242.56502854685928,53.667752824907645,1.0,0.0034401541416594996,0.6700000312414343,1.0,0.0,34061.20830910968,20.178575141850796,nan,0.1099660983385235
1084427165761544,242.565058139153,53.6660585113414,10153,sv3,0.125989095083328,3.5227684897735e-05,4,5.68882738612592,fuji/healpix/sv3/dark/101/10153/redrock-sv3-dark-10153.fits,TFT,123,242.56505813915317,53.66605851134143,1.0,0.0051345584203787275,1.0000000466273342,1.0,0.0,4803.573578650942,10.904640688244093,nan,0.1099660983385235
39633304835591852,242.564968458214,53.671192794853,10153,sv3,0.109965464429358,1.30236298072821e-05,0,847.705579817295,fuji/healpix/sv3/bright/101/10153/redrock-sv3-bright-10153.fits,TFT,123,242.56496845821405,53.67119279485298,1.0,1.6837541040866038e-14,3.2792580096330368e-12,1.0,0.0,-0.19004118687180754,4.756134887529947,nan,0.1099660983385235
39633304835591852,242.564968458214,53.671192794853,10153,sv3,0.109966732247689,1.26106529808024e-05,0,879.694150358438,iron/healpix/sv3/bright/101/10153/redrock-sv3-bright-10153.fits,TFT,123,242.56496845821405,53.67119279485298,1.0,1.6837541040866038e-14,3.2792580096330368e-12,1.0,0.0,0.190041186875968,4.655036418078015,nan,0.1099660983385235
